### This notebook helps playing wordle by suggesting the best guesses after the first guess

In [1]:
import numpy as np
import random
import math
import time
import re
import string
from tqdm.notebook import tqdm_notebook

from itertools import permutations,combinations_with_replacement,combinations

In [2]:
#get a list of all lowercase letters of the alphabet to check
letters = list(string.ascii_lowercase)
len(letters)

26

In [3]:
#get all the 12972 possible words that can be guessed
guesses = open('/content/Worlde_Word_List.txt','r').read().splitlines()
print(len(guesses))
#print(guesses[:10])
TOTAL_WORDS=len(guesses)

12972


In [4]:
#get a dictionary with all words along with their probability of occurences

long_freqs = open('/content/long_freqs.txt','r').read().splitlines() #12972 words
short_freqs = open('/content/short_freqs.txt','r').read().splitlines() #2315 words 

print(len(long_freqs),len(short_freqs))
#print(len(long_freqs)+len(short_freqs))
freqs = long_freqs + short_freqs
#print(len(freqs))

freqs_dict={}
for item in freqs:
  x = item.split(' ')
  freqs_dict[x[0]] = float(x[1])
print(len(freqs_dict))
del long_freqs, short_freqs, #freqs

10657 2315
12972


**This function eliminates words based on the word we gussed that is the occurance of the letters in grey, yellow and the greens**

In [5]:
def eliminate_words(not_letters, present_letters,pos_letters,pos_absent):
  letter_counter = 0
  remain = []

  for item in pos_letters:
    if item in letters:#letters = all 26 letters
      letter_counter +=1
  #print('letter_counter:',letter_counter)
  #letter_counter keeps counts of how many letters are in the fixed position

      
  #print(not_letters)
  for item in guesses:
    temp = list(item)
    #print(temp)
    #eliminates letters that aren't present
    if ((len((set(temp) & set(not_letters)))) == 0) and\
                                      ((len((set(temp) & set(present_letters)))) == len(present_letters)): #keeps letters are
      if letter_counter!=0:
        counter = letter_counter
        five_counter = 0
        while five_counter <5:
          current_letter = pos_letters[five_counter]
          if current_letter in letters:
            #print('here2',current_letter,temp[five_counter])

            if temp[five_counter] == current_letter:
              #print('here3',item)
              counter -=1
              five_counter +=1
              #print(counter,five_counter)
              if (counter == 0) or (five_counter == 5) :
                remain.append(item)
            else:
              break #since if one letter doesn't match, it doesn't matter if others match
          else:
            five_counter+=1
            
      else:#letter_counter !=0
        remain.append(item)

  #eliminate letter based on yellow positions
  new_remain=[]
  do_eliminate=False
  for x in pos_absent:
    if len(list(x))!=0:
      do_eliminate=True
      break

  
  if do_eliminate:
    
    for word in remain:
      match=False
      list_word = list(word)
      for indx, pos in enumerate(pos_absent):
        if match:
          break
        pos_list = list(pos)
        if len(pos_list) !=0:
          #county=len(pos_list)
          for letter in pos_list:
            if list_word[indx]==letter:
              match=True
              break
        if not match and indx==4:
          new_remain.append(word)
    return new_remain
  
  else:
    return remain

**This function calculates the information gain of each word and then sorts a dictionary with the highest information gained word at the top**

In [6]:
def get_entropy(remain,freqs,combs):
  bit_info={}
  bit_relv={}

  prob_average=[]
  for item in tqdm_notebook(remain):
    temp = list(item)
    #eliminates letters that aren't present
    probs_list=[]
    rel_probs=[]
    EI_list=[]
    EI_relv=[]
    for comb in list(combs): #go though all combinations of grey,yellow & green
      
      not_letters = []
      present_letters = []
      pos_letters = []
      letter_counter=0
      remain_words=[]
      for indx,comb_item in enumerate(comb):
        #counterss +=1
        #print(indx,comb_item)
        if comb_item == 'green':
          pos_letters.append(temp[indx])
        elif comb_item == 'yellow':
          present_letters.append(temp[indx])
          pos_letters.append('-')
        else: #comb_item == grey
          not_letters.append(temp[indx])
          pos_letters.append('-')
      #print(word,pos_letters,present_letters,not_letters)
      for ltr in pos_letters:
        if ltr in letters:#letters = all 26 letters
          letter_counter +=1
      
      for word in remain:                        
        
        temp1 = list(word)
        if ((len((set(temp1) & set(not_letters)))) == 0) and\
                                          ((len((set(temp1) & set(present_letters)))) == len(present_letters)): #keeps letters are
          if letter_counter!=0:
            counter = letter_counter
            five_counter = 0
            while five_counter <5:
              current_letter = pos_letters[five_counter]
              if current_letter in letters:
                #print('here2',current_letter,temp[five_counter])

                if temp1[five_counter] == current_letter:
                  #print('here3',item,word)
                  counter -=1
                  five_counter +=1
                  #print(counter,five_counter)
                  if (counter == 0) or (five_counter == 5) :
                    remain_words.append(word)
                else:
                  #pass
                  break #since if one letter doesn't match, it doesn't matter if others match
              else:
                five_counter+=1
                
          else:#letter_counter !=0
            
            remain_words.append(word)
      if len(remain_words)!=0:
        relv_sums =0
        for wordss in remain_words:
          relv_prob = freqs[wordss]
          #if relv_prob !=0 and relv_prob>0:
          #relvs = math.log2(1/relv_prob)
          relv_sums += relv_prob* math.log2(1/relv_prob)


        probs = len(remain_words)/len(remain)
        log2 = math.log2(1/probs)
        probs_list.append(probs)
        EI_list.append(probs*log2)
        EI_relv.append(relv_sums)

    bit_info[item]=sum(EI_list)
    bit_relv[item] = sum(EI_relv)
  
    
  
  norm_sum = np.sum(np.asarray(list(bit_relv.values())))
  norm_freqs = {key:bit_relv[key]/norm_sum for key in bit_relv.keys()}
    
  sorted_entropy = sorted(bit_info.items(), key=lambda kv: kv[1],reverse=True)
  sorted_entropy_relv = sorted(norm_freqs.items(), key=lambda kv: kv[1],reverse=True)
  return sorted_entropy,sorted_entropy_relv

**This uses permutations and combinations to capture all the diiferent ways in which green,grey and yellow letters can be arranged for a word**

In [7]:
combx = combinations_with_replacement(['grey', 'yellow', 'green'], 3)
#print(len(list(combx)))

PERMS=[]
for item in list(combx):
  x = list(permutations(item, 3))
  #for y in x:
    #if y not in PERMS:
  PERMS.extend(x)
#print(len(PERMS))
temp=[]
COMBS=[]
for permu in PERMS:
  combs = list(combinations_with_replacement(permu, 5))
  temp.extend(combs)
for item in temp:
  if item not in COMBS:
    COMBS.append(item)
print('TOTAL COMBINATIONS:',len(COMBS))

TOTAL COMBINATIONS: 99


In [8]:
# There are 70 words that have '-1' in the probability score, and we change to  a very low number 1e-16,
#so that it doesn't throw an error
counter =0
for item in freqs_dict.items():
  itm = list(item)
  if itm[-1] == -1.:
    freqs_dict[itm[0]] = 1e-16
    counter +=1
print(counter)

70


## RUN 1

In [17]:
#if one wants a random word from the whole set to make a guess in the game
print(random.choice(guesses))

means


##### This first run is after you have made the first guess and the resulting details are entered

In [18]:
#RUN 1, after the first guess is made
no_runs = 1
reduction=[TOTAL_WORDS,TOTAL_WORDS] #keeps track of the reducing number of words
present_bits = math.log2(TOTAL_WORDS)
print('Highest number of {} bits in the distribution'.format(present_bits))


guessed_word = 'means' #The word you guessed 


absent = 'mans'  #all letters in grey; the letters that aren't in the answer
present = 'e'    #letters in yellow; letters present but in the wrong position

#letters in green; here the letters have to entered accoring to their postions
#example if we get k as green in the 1st position, we write k----
#position_present
pos_present = '-----'  #there must 5 characters at all times, or an error will show up

#This is to keep tab of the letters in yellow, 
#enter the letters in yellow based on position, so words with those letters in that position aren't taken
#position_absent
pos_absent = ['','e','','','']



print('Guessed Word:',guessed_word)

if len(list(pos_present)) ==5:
  remain_words = eliminate_words(list(absent), list(present), list(pos_present),pos_absent)
  reduction.append(len(remain_words))
  print('Remaining_words:{}, {}%'.format
        (len(remain_words),(len(remain_words)/TOTAL_WORDS)*100.))
  #print("Word space reduced to {}%:".format((len(remain_words)/TOTAL_WORDS)*100.))
  sorted_entropy1,sorted_relv_entropy1 = get_entropy(remain_words,freqs_dict,COMBS)
  #print(len(sorted_entropy))
  print(sorted_entropy1[:10])
  print(sorted_relv_entropy1[:10])
  probs = len(remain_words)/reduction[no_runs]
  new_entropy = math.log2(1/probs)
  print('Number of bits droppped in entropy',new_entropy)
  print('Entropy is now reduced from {} bits to {} bits'.
        format(present_bits,present_bits-new_entropy))
  present_bits = present_bits-new_entropy
  print('Full entropy now stands at:',present_bits)

else:
  print('ERROR: lenght of pos_present should be equal to 5')

#no_runs +=1
#print(prev_word_count)

Highest number of 13.663113308455806 bits in the distribution
Guessed Word: means
Remaining_words:1082, 8.34104224483503%


  0%|          | 0/1082 [00:00<?, ?it/s]

[('puled', 5.490692730735842), ('cured', 5.483017429104285), ('fired', 5.4780091424803), ('pured', 5.467283087402873), ('lured', 5.438579121106101), ('pried', 5.421166528245736), ('tired', 5.413775180547652), ('biled', 5.3892992571870915), ('tried', 5.384607022967506), ('filed', 5.373606075042547)]
[('their', 0.0036804235124750467), ('other', 0.003613049119289145), ('there', 0.0025217716376490424), ('theic', 0.002446213440505754), ('thelf', 0.002207971208012097), ('theow', 0.002136981269220191), ('thewy', 0.002112022922341415), ('ither', 0.002110318505451857), ('cyder', 0.0019965189673968255), ('cyber', 0.001966196377267019)]
Number of bits droppped in entropy 3.583628524628991
Entropy is now reduced from 13.663113308455806 bits to 10.079484783826814 bits
Full entropy now stands at: 10.079484783826814


## RUN 2

#### This runs after the second guess is made and the resulting grey, yellow and green letters are added to eliminate more words 

In [19]:
#RUN 2, after the second guess is made

guessed_word = 'puled'


#This must include absent letters from the previous runs, because it looks all words again
absent = 'manspuld'
present = ''


pos_present = '---e-'

pos_absent = ['','','','','']


guess=[]
for indx,item in enumerate(sorted_entropy1):
  if item[0]==guessed_word:
    guess=list(sorted_entropy1[indx])

#guess = list(sorted_entropy[0])
print('Gussed Word:',guess)
claim_entropy = guess[-1]
#print('Claimed reduction in entropy:',present_bits-claim_entropy)
no_runs = 2


if len(list(pos_present)) ==5:
  remain_words = eliminate_words(list(absent), list(present), list(pos_present),pos_absent)
  reduction.append(len(remain_words))
  print('Remaining_words:{}, {}%'.format
        (len(remain_words),(len(remain_words)/TOTAL_WORDS)*100.))
  #print("Word space reduced to {}%:".format((len(remain_words)/TOTAL_WORDS)*100.))
  sorted_entropy2,sorted_relv_entropy2 = get_entropy(remain_words,freqs_dict,COMBS)
  #print(len(sorted_entropy))
  print(sorted_entropy2[:30])
  print(sorted_relv_entropy2[:30])
  probs = len(remain_words)/reduction[no_runs]
  new_entropy = math.log2(1/probs)
  
  print('Number of bits droppped in entropy',new_entropy)

  print('Claimed reduction in entropy from {} to {}:'.
        format(present_bits,present_bits-claim_entropy))
  print('Actual reduction in entropy {} bits to {} bits'.
        format(present_bits,present_bits-new_entropy))
  present_bits = present_bits-new_entropy
  print('Full entropy now stands at:',present_bits)

else:
  print('ERROR: lenght of pos_present should be equal to 5')

#no_runs +=1
#print(prev_word_count)

Gussed Word: ['puled', 5.490692730735842]
Remaining_words:153, 1.179463459759482%


  0%|          | 0/153 [00:00<?, ?it/s]

[('fiver', 8.873645816478431), ('cover', 8.476470703007939), ('wiver', 8.278054368118886), ('biter', 8.23665409056855), ('citer', 8.231681298511688), ('coyer', 8.222049491358376), ('hiver', 8.207847592854018), ('tower', 8.2072484096176), ('fiber', 8.133810205208965), ('twier', 8.096465273410514), ('twoer', 8.068005575857484), ('giver', 8.04211335675862), ('cower', 8.039603075057283), ('foyer', 8.030118850757413), ('toker', 8.008604866930888), ('bower', 7.936087502233893), ('toyer', 7.920556965889957), ('jiver', 7.889655824930909), ('kiter', 7.811672515408454), ('woker', 7.734951643715284), ('vower', 7.576274979324862), ('voter', 7.546312822267148), ('hover', 7.540978169465025), ('twyer', 7.50149456881646), ('biker', 7.3902284540868175), ('icker', 7.324479821504576), ('giber', 7.2646258424186785), ('yoker', 7.260546890336524), ('hiker', 7.210140574748124), ('joker', 7.115730960576237)]
[('other', 0.06008615512154091), ('ither', 0.02456978782924971), ('ether', 0.017453110090568213), ('gi

## RUN 3

In [20]:
#RUN 3, after the second guess is made
no_runs = 3


guessed_word = 'fiver'


absent = 'manspuldfiv'
present = ''
pos_present = '---er'

pos_absent = ['','','','','']



guess=[]
for indx,item in enumerate(sorted_entropy2):
  if item[0]==guessed_word:
    guess=list(sorted_entropy2[indx])

#guess = list(sorted_entropy[0])
print('GUESS:',guessed_word)
claim_entropy = guess[-1]
#print('Claimed reduction in entropy:',present_bits-claim_entropy)



if len(list(pos_present)) ==5:
  remain_words = eliminate_words(list(absent), list(present), list(pos_present),pos_absent)
  reduction.append(len(remain_words))
  print('Remaining_words:{}, {}%'.format
        (len(remain_words),(len(remain_words)/TOTAL_WORDS)*100.))
  #print("Word space reduced to {}%:".format((len(remain_words)/TOTAL_WORDS)*100.))
  sorted_entropy3,sorted_relv_entropy3 = get_entropy(remain_words,freqs_dict,COMBS)
  #print(len(sorted_entropy))
  print(sorted_entropy3[:10])
  print(sorted_relv_entropy3[:10])
  probs = len(remain_words)/reduction[no_runs]
  new_entropy = math.log2(1/probs)
  
  print('Number of bits droppped in entropy',new_entropy)

  print('Claimed reduction in entropy from {} to {}:'.
        format(present_bits,present_bits-claim_entropy))
  print('Actual reduction in entropy {} bits to {} bits'.
        format(present_bits,present_bits-new_entropy))
  present_bits = present_bits-new_entropy
  print('Full entropy now stands at:',present_bits)

else:
  print('ERROR: lenght of pos_present should be equal to 5')

#no_runs +=1
#print(prev_word_count)

GUESS: fiver
Remaining_words:40, 0.3083564600678384%


  0%|          | 0/40 [00:00<?, ?it/s]

[('tower', 11.739108429853665), ('cower', 11.370764993284874), ('toker', 11.133960323522068), ('toyer', 11.09071694844611), ('twyer', 10.918274404809186), ('coyer', 10.874866964481855), ('twoer', 10.839894280839518), ('woker', 10.674566092387831), ('bower', 10.603773194888111), ('other', 9.691834511140584)]
[('other', 0.3071857137013936), ('ether', 0.08615827589389365), ('cyber', 0.05807805569921297), ('ocher', 0.057399432694330545), ('ocker', 0.0568030419634055), ('otter', 0.0564642789901082), ('tower', 0.033836585361398254), ('toker', 0.029451033337051857), ('toyer', 0.029385098911354754), ('borer', 0.02933479982621501)]
Number of bits droppped in entropy 1.9354597478052893
Claimed reduction in entropy from 7.25738784269265 to -1.616257973785781:
Actual reduction in entropy 7.25738784269265 bits to 5.321928094887361 bits
Full entropy now stands at: 5.321928094887361


## RUN 4

In [22]:
#RUN 4, after the second guess is made
no_runs = 4


guessed_word = 'tower'


absent = 'manspuldfivw'
present = 'to'
pos_present = '---er'

pos_absent = ['t','o','','','']



guess=[]
for indx,item in enumerate(sorted_entropy3):
  if item[0]==guessed_word:
    guess=list(sorted_entropy3[indx])

#guess = list(sorted_entropy[0])
print('GUESS:',guess)
claim_entropy = guess[-1]

if len(list(pos_present)) ==5:
  remain_words = eliminate_words(list(absent), list(present), list(pos_present),pos_absent)
  reduction.append(len(remain_words))
  print('Remaining_words:{}, {}%'.format
        (len(remain_words),(len(remain_words)/TOTAL_WORDS)*100.))
  #print("Word space reduced to {}%:".format((len(remain_words)/TOTAL_WORDS)*100.))
  sorted_entropy4,sorted_relv_entropy4 = get_entropy(remain_words,freqs_dict,COMBS)
  #print(len(sorted_entropy))
  print(sorted_entropy4[:10])
  print(sorted_relv_entropy4[:10])
  probs = len(remain_words)/reduction[no_runs]
  new_entropy = math.log2(1/probs)
  
  print('Number of bits droppped in entropy',new_entropy)

  print('Claimed reduction in entropy from {} to {}:'.
        format(present_bits,present_bits-claim_entropy))
  print('Actual reduction in entropy {} bits to {} bits'.
        format(present_bits,present_bits-new_entropy))
  present_bits = present_bits-new_entropy
  print('Full entropy now stands at:',present_bits)

else:
  print('ERROR: lenght of pos_present should be equal to 5')

#no_runs +=1
#print(prev_word_count)

GUESS: ['tower', 11.739108429853665]
Remaining_words:3, 0.023126734505087884%


  0%|          | 0/3 [00:00<?, ?it/s]

[('other', 13.459650006730794), ('oxter', 13.459650006730794), ('otter', 6.818045838862195)]
[('other', 0.7849239359206597), ('otter', 0.1435079739526842), ('oxter', 0.07156809012665614)]
Number of bits droppped in entropy 3.736965594166206
Claimed reduction in entropy from 5.321928094887361 to -6.417180334966305:
Actual reduction in entropy 5.321928094887361 bits to 1.5849625007211547 bits
Full entropy now stands at: 1.5849625007211547


## RUN 5

In [40]:
#RUN 4, after the second guess is made
no_runs = 5


guessed_word = 'tafia'


absent = 'drpeolsnkygf'
present = ''
pos_present = 'ta-i-'

pos_absent = ['','','','','']



guess=[]
for indx,item in enumerate(sorted_entropy4):
  if item[0]==guessed_word:
    guess=list(sorted_entropy4[indx])

#guess = list(sorted_entropy[0])
print('GUESS:',guess)
claim_entropy = guess[-1]

if len(list(pos_present)) ==5:
  remain_words = eliminate_words(list(absent), list(present), list(pos_present),pos_absent)
  reduction.append(len(remain_words))
  print('Remaining_words:{}, {}%'.format
        (len(remain_words),(len(remain_words)/TOTAL_WORDS)*100.))
  #print("Word space reduced to {}%:".format((len(remain_words)/TOTAL_WORDS)*100.))
  sorted_entropy5,sorted_relv_entropy5 = get_entropy(remain_words,freqs_dict,COMBS)
  #print(len(sorted_entropy))
  print(sorted_entropy5[:10])
  print(sorted_relv_entropy5[:10])
  probs = len(remain_words)/reduction[no_runs]
  new_entropy = math.log2(1/probs)
  
  print('Number of bits droppped in entropy',new_entropy)

  print('Claimed reduction in entropy from {} to {}:'.
        format(present_bits,present_bits-claim_entropy))
  print('Actual reduction in entropy {} bits to {} bits'.
        format(present_bits,present_bits-new_entropy))
  present_bits = present_bits-new_entropy
  print('Full entropy now stands at:',present_bits)

else:
  print('ERROR: lenght of pos_present should be equal to 5')

#no_runs +=1
#print(prev_word_count)

GUESS: ['tafia', 9.409022919431095]
Remaining_words:1, 0.00770891150169596%


  0%|          | 0/1 [00:00<?, ?it/s]

[('tacit', 0.0)]
[('tacit', 1.0)]
Number of bits droppped in entropy 2.321928094887362
Claimed reduction in entropy from -20.51159449719137 to -29.920617416622466:
Actual reduction in entropy -20.51159449719137 bits to -22.833522592078733 bits
Full entropy now stands at: -22.833522592078733


## END